# Theoretical Neuroscience: Project 1

**Professor:** Brian Frost

**Spring 2025**

---

This notebook implements spike‐train generation using concepts from Chapter 1 of _Dayan and Abbott’s Theoretical Neuroscience_.

We consider:

- **Homogeneous Poisson Process:** Spike trains with a constant firing rate

- **Inhomogeneous Poisson Process:** Spike trains with a time‐varying firing rate

- **Refractory Period:** After a spike the neuron is “blocked” for an interval drawn from a hard‐coded (exponential) distribution

Computations for each process (with and without refractoriness):

- The **Fano factor** (variance/mean of spike counts over many trials)

- The **interspike interval (ISI) probability distributions**

- The **coefficient of variation (CV)** of the ISIs

- The **spike‐train autocorrelation function**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Simulation Parameters

Setting each trial to be 1 second long, the spike train is represented in an array with each value representing a 1 ms time bin.

In [2]:
T = 1 # Trial duration (s)
dt = 0.001 # Bin size (s)
# time = np.arange(0, T, dt)

## Spike Train Generation

**Poisson Process**

The poisson distribution is given by

$$
P_T[n] = \frac{(rT)^n}{n!} \exp(-rT).
$$

where $P_T[n]$ is the probability of observing $n$ spikes in time $T$ given a firing rate of $r$. For a homogeneous poisson process, the firing rate is constant; for an inhomogeneous poisson process, the firing rate is time-varying.

For the computational generation I used the following procedure:

> "Spike sequences can be simulated by using some estimate of the firing rate, $r_{\text{est}}(t)$, predicted from knowledge of the stimulus, to drive a Poisson process. A simple procedure for generating spikes in a computer program is based on the fact that the estimated probability of firing a spike during a short interval of duration $\Delta t$ is $r_{\text{est}}(t) \Delta t$. The program progresses through time in small steps of size $\Delta t$ and generates, at each time step, a random number $x*{\text{rand}}$ chosen uniformly in the range between 0 and 1. If $r_{\text{est}}(t) \Delta t > x\_{\text{rand}}$ at that time step, a spike is fired; otherwise it is not." ({cite}`dayan_theoretical_2001`, p.47)

**Refractory Period**

> "For a few milliseconds just after an action potential has been fired, it may be virtually impossible to initiate another spike. This is called the absolute refractory period." ({cite}`dayan_theoretical_2001`, p.21)

> "Refractory effects can be incorporated into a Poisson model of spike generation by setting the firing rate to 0 immediately after a spike is fired, and then letting it return to its predicted value according to some dynamic rule such as an exponential recovery." ({cite}`dayan_theoretical_2001`, p.48)

> "Both the length of the absolute and relative refractory period depend on the nerve cell under consideration, but typical values in the central nervous system are 0.5–1 ms and ≈ 10 ms, respectively." {cite}`gabbiani_mathematics_2010`

The rule I implemented is an absolute refractory period of 1 ms summed with a relative refractory period drawn from an exponential distribution with a mean of 10 ms.
The exponential distribution is given by

$$
f(x; \frac{1}{\beta}) = \frac{1}{\beta} \exp(-\frac{x}{\beta}),
$$

where $\beta$ is the mean of the distribution.


In [3]:
# Initialize random number generator
rng = np.random.default_rng()

### Homogeneous Poisson Process

In [4]:
def generate_spike_train_homogeneous(
    r: int,
    T: float,
    dt: float,
    refractory: bool = False,
    abs_ref: float = 0.001,
    rel_ref_mean: float = 0.01,
) -> np.ndarray:
    """Generate a spike train array according to an homogeneous Poisson process with rate _r_

    Args:
        r (int): firing rate (Hz)
        T (float): trial duration (s)
        dt (float): time bin width (s)
        refractory (bool, optional): whether to include refractory periods. Defaults to False.
        abs_ref (float, optional): absolute refractory period (s). Defaults to 0.001.
        rel_ref_mean (float, optional): mean relative refractory period (s). Defaults to 0.01.

    Returns:
        np.ndarray: spike train array; when a spike is present, the value is 1, otherwise 0
    """
    spike_train_length = int(T/dt)
    spike_train = np.zeros(spike_train_length)
    next_possible_index = 0  # index until which the neuron is refractory

    for i in range(spike_train_length):
        if i < next_possible_index:
            continue
        # The probability to spike in a bin is rate*dt.
        if rng.random() < r * dt:
            spike_train[i] = 1
            if refractory:
                ref_period = abs_ref + rng.exponential(scale=rel_ref_mean)
                # Convert relative refractory period to number of bins
                ref_bins = int(np.ceil(ref_period / dt))
                next_possible_index = i + ref_bins
    return spike_train
generate_spike_train_homogeneous(60, T, dt, refractory=True, rel_ref_mean=0.01)[:10]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

### Inhomogeneous Poisson Process

In [5]:
def generate_spike_train_inhomogeneous(
    r: int,
    T: float,
    dt: float,
    refractory: bool = False,
    abs_ref: float = 0.001,
    rel_ref_mean: float = 0.01,
) -> np.ndarray:
    """Generate a spike train array according to an inhomogeneous Poisson process with a time varying rate _r_

    Args:
        r (np.ndarray): array of firing rates at each time bin (Hz)
        T (float): trial duration (s)
        dt (float): time bin width (s)
        refractory (bool, optional): whether to include refractory periods. Defaults to False.
        abs_ref (float, optional): absolute refractory period (s). Defaults to 0.001.
        rel_ref_mean (float, optional): mean relative refractory period (s). Defaults to 0.01.

    Returns:
        np.ndarray: spike train array; when a spike is present, the value is 1, otherwise 0
    """
    spike_train_length = int(T/dt)
    spike_train = np.zeros(spike_train_length)
    next_possible_index = 0  # index until which the neuron is refractory

    for i in range(spike_train_length):
        if i < next_possible_index:
            continue
        # The probability to spike in a bin is rate*dt.
        if rng.random() < r[i] * dt:
            spike_train[i] = 1
            if refractory:
                ref_period = abs_ref + rng.exponential(scale=rel_ref_mean)
                # Convert relative refractory period to number of bins
                ref_bins = int(np.ceil(ref_period / dt))
                next_possible_index = i + ref_bins
    return spike_train
generate_spike_train_inhomogeneous(np.full(int(T/dt), 60), T, dt, refractory=True, rel_ref_mean=0.01)[:10]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

```{bibliography}
```